# Project Statistical Analysis

Karla y Mar

*Data Part Time Barcelona Dic 2019*

<img src="./img/5530085.jpeg" width="1000"> 

# 1. Importamos librería

In [3]:
import pandas as pd
import chart_studio.plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import cufflinks as cf
from datetime import datetime
from ipywidgets import interact
import ipywidgets as wdg
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import warnings
import cartopy
warnings.filterwarnings('ignore')
import plotly.offline as off
import json
import requests
from pandas.io.json import json_normalize
import re
cf.go_offline()

# 2. Importamos datos

## 2.1. API datos coronavirus a nivel mundial

In [99]:
url='https://api.covid19api.com/all'
response = requests.get(url, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"})
result=response.json()
df_coronavirus_raw = pd.DataFrame()
df_coronavirus_raw = json_normalize(result)
df_coronavirus_raw.head()

,Country,Province,Lat,Lon,Date,Cases,Status
0,Azerbaijan,,0.0,0.0,2020-02-28T00:00:00Z,1,confirmed
1,Azerbaijan,,0.0,0.0,2020-02-28T00:00:00Z,0,recovered
2,Azerbaijan,,0.0,0.0,2020-02-28T00:00:00Z,0,deaths
3,Afghanistan,,33.0,65.0,2020-02-24T00:00:00Z,1,confirmed
4,Afghanistan,,33.0,65.0,2020-02-24T00:00:00Z,0,recovered


## 2.2. Continents by country

In [74]:
df_continents = pd.read_csv('./data/concap.csv')
df_continents.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe


## 2.3. Join corinavirus info and continent info

In [75]:
df_coronavirus_raw = df_coronavirus_raw.merge(df_continents, left_on='Country', right_on='CountryName')

## 2.4. Regional information

In [106]:
url='https://api.covid19tracking.narrativa.com/api?date_from=2020-03-10&date_to=2020-03-17'
response = requests.get(url, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"})
result=response.json()
result

{'dates': {'2020-03-10': {'countries': {'Afghanistan': {'name': 'Afghanistan',
     'name_es': 'Afganistán',
     'regions': [],
     'today_confirmed': 5,
     'today_deaths': 0,
     'today_new_confirmed': 1,
     'today_new_deaths': 0,
     'today_new_open_cases': 1,
     'today_new_recovered': 0,
     'today_open_cases': 5,
     'today_recovered': 0,
     'today_vs_yesterday_confirmed': 0.25,
     'today_vs_yesterday_deaths': None,
     'today_vs_yesterday_open_cases': 0.25,
     'today_vs_yesterday_recovered': None,
     'yesterday_confirmed': 4,
     'yesterday_deaths': 0,
     'yesterday_open_cases': 4,
     'yesterday_recovered': 0},
    'Albania': {'name': 'Albania',
     'name_es': 'Albania',
     'regions': [],
     'today_confirmed': 10,
     'today_deaths': 0,
     'today_new_confirmed': 8,
     'today_new_deaths': 0,
     'today_new_open_cases': 8,
     'today_new_recovered': 0,
     'today_open_cases': 10,
     'today_recovered': 0,
     'today_vs_yesterday_confirmed': 4

In [144]:
result.keys()

dict_keys(['dates', 'metadata', 'total', 'updated_at'])

In [118]:
df_regions_raw = pd.DataFrame()
# df_coro_reg_raw = json_normalize(json_normalize(result).dates[0])
df_regions_raw = json_normalize(result)
# df_regions_raw.columns=[str.strip(x.lower()) for x in dfArticles.columns]
df_regions_raw.head()


,updated_at,dates.2020-03-10.countries.Afghanistan.name,dates.2020-03-10.countries.Afghanistan.name_es,dates.2020-03-10.countries.Afghanistan.regions,dates.2020-03-10.countries.Afghanistan.today_confirmed,dates.2020-03-10.countries.Afghanistan.today_deaths,dates.2020-03-10.countries.Afghanistan.today_new_confirmed,dates.2020-03-10.countries.Afghanistan.today_new_deaths,dates.2020-03-10.countries.Afghanistan.today_new_open_cases,dates.2020-03-10.countries.Afghanistan.today_new_recovered,...,total.today_open_cases,total.today_recovered,total.today_vs_yesterday_confirmed,total.today_vs_yesterday_deaths,total.today_vs_yesterday_open_cases,total.today_vs_yesterday_recovered,total.yesterday_confirmed,total.yesterday_deaths,total.yesterday_open_cases,total.yesterday_recovered
0,2020-03-24 12:50UTC,Afghanistan,Afganistán,[],5,0,1,0,1,0,...,99708,79737,0.095441,0.127246,0.152361,0.02905,170911,6900,86525,77486


In [141]:
#Datos de España
path = './data/data_output_covid19-cases-uci-deaths-by-ccaa-spain-by-day-accumulated.csv'
df_spain_deceased = pd.read_csv(path, sep=',')
df_spain_deceased.head()

,date,region_code,region,country,population,cases_registered,cases_per_100000,intensive_care,intensive_care_per_1000000,deceassed,deceassed_per_100000,recovered,recovered_per_100000
0,2020-02-27,1,Andalucía,Spain,8414240,1,0.01,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-28,1,Andalucía,Spain,8414240,6,0.07,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-29,1,Andalucía,Spain,8414240,8,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-01,1,Andalucía,Spain,8414240,12,0.14,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-02,1,Andalucía,Spain,8414240,12,0.14,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
#Datos de España, Italia y Francia
path = './data/data_output_covid19-countries-regions-compile.csv'
df_sp_it_fr_conf = pd.read_csv(path, sep=',')
df_sp_it_fr_conf.head()

,date,region_code,region,country,population,cases_registered,cases_per_100000,intensive_care,intensive_care_per_1000000,deceassed,deceassed_per_100000,recovered,recovered_per_100000
0,2020-02-27,1.0,Andalucía,Spain,8414240.0,1,0.01,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-28,1.0,Andalucía,Spain,8414240.0,6,0.07,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-29,1.0,Andalucía,Spain,8414240.0,8,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-01,1.0,Andalucía,Spain,8414240.0,12,0.14,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-02,1.0,Andalucía,Spain,8414240.0,12,0.14,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
#Datos de camas UCI
path = './data/ccaa_camas_uci_2017.csv'
df_camas_uci = pd.read_csv(path, sep=',')
df_camas_uci.head()

,cod_ine,CCAA,Públicos,Privados,Total
0,1,ANDALUCÍA,572,162,734
1,2,ARAGÓN,113,22,135
2,3,ASTURIAS,86,7,93
3,15,C. FORAL DE NAVARRA,46,19,65
4,5,CANARIAS,187,50,237


In [146]:
#Datos de mascarillas entregadas
path = './data/ccaa_covid19_mascarillas.csv'
df_camas_uci = pd.read_csv(path, sep=',')
df_camas_uci.head()

,fecha,cod_ine,CCAA,mascarillas_acumulado_desde_2020-03-10
0,2020-03-22,1.0,ANDALUCÍA,444083
1,2020-03-22,2.0,ARAGÓN,72455
2,2020-03-22,3.0,ASTURIAS,60229
3,2020-03-22,4.0,BALEARES,49476
4,2020-03-22,5.0,CANARIAS,124165


In [147]:
#Datos por rangos de edad
path = './data/nacional_covid19_rango_edad.csv'
df_camas_uci = pd.read_csv(path, sep=',')
df_camas_uci.head()

,fecha,rango_edad,sexo,casos_confirmados,hospitalizados,ingresos_uci,fallecidos
0,2020-03-23,0-9,ambos sexos,129,34,1,0
1,2020-03-23,10-19,ambos sexos,221,15,0,1
2,2020-03-23,20-29,ambos sexos,1285,183,8,4
3,2020-03-23,30-39,ambos sexos,2208,365,15,3
4,2020-03-23,40-49,ambos sexos,2919,663,40,9


# 3. Data cleaning

## 3.1. Cases by country

### 3.1.1. Renombramos columnas y estandarizamos nombres

In [76]:
df_coronavirus_raw.columns = df_coronavirus_raw.columns.str.replace('[.]','_').str.lower()
df_coronavirus_raw = df_coronavirus_raw.rename(columns = {'lat':'latitude', 'lon':'longitude', 'capitalname':'capital', 
                                                         'capitallatitude':'capital_latitude', 'capitallongitude':'capital_longitude',
                                                         'countrycode':'country_code', 'continentname':'continent'})
df_coronavirus_raw.head()

,country,province,latitude,longitude,date,cases,status,countryname,capital,capital_latitude,capital_longitude,country_code,continent
0,Afghanistan,,33.0,65.0,2020-02-24T00:00:00Z,1,confirmed,Afghanistan,Kabul,34.516667,69.183333,AF,Asia
1,Afghanistan,,33.0,65.0,2020-02-24T00:00:00Z,0,recovered,Afghanistan,Kabul,34.516667,69.183333,AF,Asia
2,Afghanistan,,33.0,65.0,2020-02-24T00:00:00Z,0,deaths,Afghanistan,Kabul,34.516667,69.183333,AF,Asia
3,Afghanistan,,0.0,0.0,2020-02-24T00:00:00Z,1,confirmed,Afghanistan,Kabul,34.516667,69.183333,AF,Asia
4,Afghanistan,,0.0,0.0,2020-02-24T00:00:00Z,0,recovered,Afghanistan,Kabul,34.516667,69.183333,AF,Asia


### 3.1.2. Eliminamos columnas duplicadas

In [77]:
df_coronavirus_raw.drop('countryname', axis=1, inplace=True)

### 3.1.3. Clasificamos los paises entre China y resto del mundo

In [78]:
df_coronavirus_raw[df_coronavirus_raw['country']=='China'].head(50)

,country,province,latitude,longitude,date,cases,status,capital,capital_latitude,capital_longitude,country_code,continent
3564,China,Tibet,31.6927,88.0924,2020-02-23T00:00:00Z,1,confirmed,Beijing,39.916667,116.383333,CN,Asia
3565,China,Tibet,31.6927,88.0924,2020-02-23T00:00:00Z,1,recovered,Beijing,39.916667,116.383333,CN,Asia
3566,China,Tibet,31.6927,88.0924,2020-02-23T00:00:00Z,0,deaths,Beijing,39.916667,116.383333,CN,Asia
3567,China,Qinghai,35.7452,95.9956,2020-02-23T00:00:00Z,18,confirmed,Beijing,39.916667,116.383333,CN,Asia
3568,China,Qinghai,35.7452,95.9956,2020-02-23T00:00:00Z,18,recovered,Beijing,39.916667,116.383333,CN,Asia
3569,China,Qinghai,35.7452,95.9956,2020-02-23T00:00:00Z,0,deaths,Beijing,39.916667,116.383333,CN,Asia
3570,China,Xinjiang,41.1129,85.2401,2020-03-08T00:00:00Z,76,confirmed,Beijing,39.916667,116.383333,CN,Asia
3571,China,Xinjiang,41.1129,85.2401,2020-03-08T00:00:00Z,73,recovered,Beijing,39.916667,116.383333,CN,Asia
3572,China,Xinjiang,41.1129,85.2401,2020-03-08T00:00:00Z,3,deaths,Beijing,39.916667,116.383333,CN,Asia
3573,China,Anhui,31.8257,117.2260,2020-03-08T00:00:00Z,990,confirmed,Beijing,39.916667,116.383333,CN,Asia


In [80]:
df_coronavirus_raw['class'] = np.where(df_coronavirus_raw.country.str.contains('China'), 'China', 'Other Countries')
df_coronavirus_raw['date'] = pd.to_datetime(df_coronavirus_raw['date'], format="%Y-%m-%dT%H:%M:%SZ")
l_index = ['class','continent','country', 'province', 'latitude','longitude', 'date']
df_coronavirus = pd.pivot_table(df_coronavirus_raw, values='cases', 
                index = l_index, columns=['status'],
                aggfunc=np.sum, fill_value=0).reset_index()
df_coronavirus.head()

status,class,continent,country,province,latitude,longitude,date,confirmed,deaths,recovered
0,China,Asia,China,Anhui,31.8257,117.226,2020-03-08,990,6,984
1,China,Asia,China,Anhui,31.8257,117.226,2020-03-11,990,6,984
2,China,Asia,China,Anhui,31.8257,117.226,2020-03-12,990,6,984
3,China,Asia,China,Anhui,31.8257,117.226,2020-03-13,990,6,984
4,China,Asia,China,Anhui,31.8257,117.226,2020-03-14,990,6,984


### 3.1.4. Obtenemos los totales por países y calculamos las estadísticas

- Los casos existentes se han calculado de la siguiente manera :    
         casos existentes = casos confirmados - fallecidos - recuperados
- La columna <CODE>'days_province_confirmed'</CODE> nos crea un contador a partir de los 100 casos confirmados.
- La columna <CODE>'estimated'</CODE> nos da una aproximación de los posibles casos que hay teniendo en cuanta la mortalidad.
         casos estimados = fallecidos * 99
- La columna <CODE>'days_ death'</CODE> nos crea un contador a partir de los 10 fallecimientos.

In [82]:
df_countries = df_coronavirus.groupby(['class','continent','country','date'])['confirmed','deaths', 'recovered'].agg('sum').reset_index()
df_countries['existing'] = df_countries.confirmed.fillna(0) -  df_countries.deaths.fillna(0) - df_countries.recovered.fillna(0)
df_countries['days_confirmed'] = df_countries[df_countries['confirmed']>100].groupby(['country']).cumcount()+1
df_countries['days_confirmed'] = df_countries['days_confirmed'].fillna(0).astype('int')
df_countries['days_death'] = df_countries[df_countries['deaths']>10].groupby(['country']).cumcount()+1
df_countries['days_death'] = df_countries['days_death'].fillna(0).astype('int')
df_countries['days_recovered'] = df_countries[df_countries['confirmed']>100].groupby(['country']).cumcount()+1
df_countries['days_recovered'] = df_countries['days_recovered'].fillna(0).astype('int')
df_countries['estimated'] = [confirmed*99 for confirmed in df_countries['confirmed']]
df_countries.head()

status,class,continent,country,date,confirmed,deaths,recovered,existing,days_confirmed,days_death,days_recovered,estimated
0,China,Asia,China,2020-02-23,19,0,19,0,0,0,0,1881
1,China,Asia,China,2020-03-08,1066,9,1057,0,1,0,1,105534
2,China,Asia,China,2020-03-11,80921,3161,61644,16116,2,1,2,8011179
3,China,Asia,China,2020-03-12,80932,3172,62901,14859,3,2,3,8012268
4,China,Asia,China,2020-03-13,80945,3180,64196,13569,4,3,4,8013555


### 3.1.5. Ordenamos columas

In [83]:
column_order = ['date', 'class', 'continent', 'country', 'confirmed','deaths', 'recovered', 'existing', 'days_confirmed', 'days_death','days_recovered']
df_countries = df_countries[column_order]
df_countries.head()

status,date,class,continent,country,confirmed,deaths,recovered,existing,days_confirmed,days_death,days_recovered
0,2020-02-23,China,Asia,China,19,0,19,0,0,0,0
1,2020-03-08,China,Asia,China,1066,9,1057,0,1,0,1
2,2020-03-11,China,Asia,China,80921,3161,61644,16116,2,1,2
3,2020-03-12,China,Asia,China,80932,3172,62901,14859,3,2,3
4,2020-03-13,China,Asia,China,80945,3180,64196,13569,4,3,4


## 3.2. Cases by regions

In [ ]:
https://api.covid19tracking.narrativa.com/api?date_from=2020-03-10&date_to=2020-03-17

# 4. Comparación de la evolución de casos entre dos países

## Diagramamos el scatterplot

In [85]:
col_vars = ['confirmed','deaths', 'recovered', 'existing']
cou_vars = list(df_countries['country'].unique())

def make_hist(country1='Italy', country2='Spain'):
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=(country1, country2))
    for x in range(0,len(col_vars)):
        df_filtered = df_countries[df_countries['country']==country1]
        fig.add_trace(go.Scatter( \
                      x = df_filtered['date'], \
                      y = df_filtered[col_vars[x]], \
                      mode = 'lines+markers',
                      name = f'{country1} - {col_vars[x].capitalize()}'),
                      row=1, col=1)
        df_filtered2 = df_countries[df_countries['country']==country2]
        fig.add_trace(go.Scatter( \
                      x = df_filtered2['date'], \
                      y = df_filtered2[col_vars[x]], \
                      mode = 'lines+markers',
                      name = f'{country2} - {col_vars[x].capitalize()}'),
                      row=1, col=2)
        
    fig.update_layout(height=500, width=1000, title_text=f'Evolución de casos {country1} vs. {country2}', title_x=0.5)
    fig.show()
                      
_= interact(make_hist, country1 = cou_vars, country2 = cou_vars)

interactive(children=(Dropdown(description='country1', index=120, options=('China', 'Algeria', 'Angola', 'Beni…

In [87]:
def make_hist(country1='Italy', country2='Spain'):
    nrows = 2
    ncols = nrows
    fig = make_subplots(rows=2, cols=2, subplot_titles=col_vars)
    for c in range(0,len(col_vars)):
        df_filtered = df_countries[df_countries['country']==country1]
        fig.add_trace(go.Scatter( \
                      x = df_filtered['days_confirmed'], \
                      y = df_filtered[col_vars[c]], \
                      mode = 'lines+markers',
                      name = f'{country1} - {col_vars[c].capitalize()}'),
                      row=int(c/nrows)+1, col=(c%ncols)+1)
        df_filtered2 = df_countries[df_countries['country']==country2]
        fig.add_trace(go.Scatter( \
                      x = df_filtered2['days_confirmed'], \
                      y = df_filtered2[col_vars[c]], \
                      mode = 'lines+markers',
                      name = f'{country2} - {col_vars[c].capitalize()}'),
                      row=int(c/nrows)+1, col=(c%ncols)+1)
        
    fig.update_layout(height=1000, width=1000, title_text=f'Evolución de casos {country1} vs. {country2}', title_x=0.5)
    fig.show()
                      
_= interact(make_hist, country1 = cou_vars, country2 = cou_vars)

interactive(children=(Dropdown(description='country1', index=120, options=('China', 'Algeria', 'Angola', 'Beni…

**Insights**: 
* Se puede ver un comportamiento distinto entre China y el resto del mundo, por lo que se han dividido las gráficas en estos dos grupos.
  * China: Un aumento muy claro de casos desde el inicio de la enfermedad hasta el 14 de febrero (con picos el 12, 13 y 14 de Febrero). A partir de allí las medidas tomadas por el gobierno de China comienzan a dar resutados y el número de casos que se recuperan sube exponencialmente. Asimismo los casos confirmados bajan diariamente lo que indica la remisión del virus. Sin embargo el virus ha dejado 3193 muetros en el territorio de China.
  * Resto del mundo: A partir del 14 de Febrero los casos en los demás paises comienzan a crecer exponencialmente. A partir del 10 de Marzo se dan crecimientos más altos que los días de crecimiento más altos de la enfermedad en China, por lo que se declara el grado de pandemia. Se puede ser además que si no se toman las medidas de contención necesarias y se sigue la misma curva de progresión de la enfermedad que en China (en el mejor de los casos) la enfermedad puede llegar a duplicar el número de casos confirmados en el mundo.

# 5. Suma acumulada de casos de coronavirus por país

##  Sumarizamos los casos para obtener los acumulados por país y por fecha

In [88]:
col_vars = ['confirmed','deaths', 'recovered', 'existing']
cou_vars = ['All']
cou_vars.extend(sorted(list(df_coronavirus['country'].unique())))

## Creamos un combobox con los países y diagramamos los casos confirmados, muertos, recuperados y existentes

In [89]:
def make_hist(Country): 
    nrows = 2
    ncols = nrows
    fig = make_subplots(rows = nrows, cols = ncols, subplot_titles=[f'Total of {c} cases' for c in col_vars])
    for c in range(0,len(col_vars)):
        if(Country=='All'):
            df_filtered = df_countries
        else:
            df_filtered = df_countries[df_countries['country']==Country]
        fig.add_trace(
            go.Bar(x = df_filtered['date'], \
                y = df_filtered[col_vars[c]], \
                name = col_vars[c].capitalize()),
                row=int(c/nrows)+1, col=(c%ncols)+1)
    fig.update_layout(height=1000, width=1000, title_text="Casos acumulados de coronavirus por país", title_x=0.5)
    fig.show()

_= interact(make_hist, Country = (cou_vars))

interactive(children=(Dropdown(description='Country', options=('All', 'Afghanistan', 'Albania', 'Algeria', 'An…

**Insights**: 
* Si seleccionamos la opción All, se puede ver el la composición de casos por país, donde hasta hace unos días el protagonista era China. A partir del 14 de Febrero la actividad del virus remite en China, pero se contafia al resto del mundo. Adicionalmente se puede ser que el ratio de muerte de la enfermedad es muy bajo.
* En España e Italia los casos han crecido exponencialmente desde Marzo.

# 6. Top 10 de los países más afectados por el coronavirus

In [90]:
df_countries_top10 = df_countries.groupby('date',group_keys=False)\
            .apply(lambda grp:grp.nlargest(n=int(10),columns='confirmed').sort_index()).reset_index()

In [91]:
options = ["Include China","Exclude China"]

def top_ten (options):
    if options == "Incluye China":
        df_countries_top10 = df_countries.groupby('date',group_keys=False)\
        .apply(lambda grp:grp.nlargest(n=int(10),columns='confirmed').sort_index()).reset_index()
    elif options == "Excluye China":       
        df_countries_top10 = df_countries[df_countries['country']!="China"].groupby('date',group_keys=False)\
            .apply(lambda grp:grp.nlargest(n=int(11),columns='confirmed').sort_index()).reset_index()
        #Con esto sacamos a China del gráfico para que no distorsione
#         df_countries_top10 = df_countries_top10[df_countries_top10['country']!="China"]

    fig =px.bar(df_countries_top10, x='confirmed', y='country',  
             labels={'confirmed': f'Confirmed cases by country',
                    'country':'Top 10 countries with confirmed cases'},
             animation_frame=df_countries_top10.date.dt.strftime('%Y-%m-%d'), orientation='h').update_yaxes(categoryorder='total ascending')
    fig.update_layout(title_text="Evolution of confirmed cases by country: Top 10", title_x=0.5)
    return fig.show()
_= interact(top_ten, options = options)

interactive(children=(Dropdown(description='options', options=('Incluye China', 'Excluye China'), value='Inclu…

**Insights:**

En esta progresión de tiempo se puede ver el crecimiento de la expansión de virus en europa especialmente en Italia y España en Europa e Iran y Corea del Sur en el mes de Marzo. Antes de ello la cantidad de casos de concentraba en China.

# 7. Casos confirmados de coronavirus: Estadísticas

In [92]:
df_countries_top10 = df_countries.groupby('date',group_keys=False)\
            .apply(lambda grp:grp.nlargest(n=int(10),columns='confirmed').sort_index()).reset_index()
df_countries_top10.head()

status,index,date,class,continent,country,confirmed,deaths,recovered,existing,days_confirmed,days_death,days_recovered
0,615,2020-01-22,Other Countries,Asia,Hong Kong,0,0,0,0,0,0,0
1,834,2020-01-22,Other Countries,Asia,Japan,2,0,0,2,0,0,0
2,996,2020-01-22,Other Countries,Asia,Macau,1,0,0,1,0,0,0
3,1416,2020-01-22,Other Countries,Asia,South Korea,1,0,0,1,0,0,0
4,1523,2020-01-22,Other Countries,Asia,Taiwan,1,0,0,1,0,0,0


In [93]:
fig = px.box(df_countries_top10, y="confirmed", x='class', 
             labels={'confirmed': f'Confirmed cases of Coronavirus',
            'class':'Agrupation by countries: China vs. Rest of the world'},
             points="all")
fig.update_layout(title_text="Stats of confirmed cases: China vs. Rest of world", title_x=0.5)
fig.show()

**Insights:**

En este gráfico no se puede sacar muchos insights ya que la curva de tiempo de la evolución del virus en China está en un estado de descenso, mientras que en el resto del mundo está ascendiendo. Lo que sí se puede ver es que la gravedad de los casos en el mundo ya está superando a China y se deben estudiar en los próximos días la evolución en los países más afectados de forma separada. 

# 8. Geoplot: Casos informados de coronavirus por países

In [97]:
col_vars = ['confirmed','deaths', 'recovered']
scope_vars = ['World', 'USA', 'Europe', 'Asia', 'Africa', 'North America', 'South America']

df_countries_region = df_coronavirus.groupby(['country','date'])[col_vars].agg('sum').reset_index()

def make_scatter(scope): 
    fig = px.scatter_geo(df_coronavirus, lon = df_coronavirus['longitude'], lat = df_coronavirus['latitude'], #color="continent",
                     scope=scope.lower(),
                     hover_name=df_coronavirus['country'], 
                     size=df_coronavirus['confirmed'],
                     size_max=50,
                     animation_frame=df_coronavirus.date.dt.strftime('%Y-%m-%d'))  
    fig.update_layout(title_text="Evolución de casos confirmados a nivel mundial", title_x=0.5, margin_l=0)
    fig.show()

_= interact(make_scatter, scope = (scope_vars))

interactive(children=(Dropdown(description='scope', options=('World', 'USA', 'Europe', 'Asia', 'Africa', 'Nort…

**Insights:**

Hace un par de meses China era el protagonista mundial de este gráfico, sin embargo el panorama actual es distinto, ya que el virus se ha diseminado por toda Europa.

# 9.Estadística

## 9.1. España vs Italia

In [ ]:
df_countries.head()

In [98]:
corona_spain = df_countries[df_countries['country'] == "Spain"]
corona_it = df_countries[df_countries['country'] == "Italy"]
corona_spain.head()

status,date,class,continent,country,confirmed,deaths,recovered,existing,days_confirmed,days_death,days_recovered
3149,2020-02-01,Other Countries,Europe,Spain,1,0,0,1,0,0,0
3150,2020-02-02,Other Countries,Europe,Spain,1,0,0,1,0,0,0
3151,2020-02-03,Other Countries,Europe,Spain,1,0,0,1,0,0,0
3152,2020-02-04,Other Countries,Europe,Spain,1,0,0,1,0,0,0
3153,2020-02-05,Other Countries,Europe,Spain,1,0,0,1,0,0,0
